## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## Step 1: Camera calibration via chessboard images. 

In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
%matplotlib qt

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images

#images=os.listdir('camera_cal')
images = glob.glob('./camera_cal/calibration*.jpg')
# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        ''' Commenting this section to stop slowing code execution during dev work. 
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        cv2.imshow('img',img)
        cv2.waitKey(500)
cv2.destroyAllWindows()'''
print('Object and Image points collected')

Object and Image points collected


#### Using the object points and image points, undistort a calibration image and check the output

In [ ]:
## Create camera calibration matrices using object and image points. 

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
## Verify capability of test matrix. 
calimg=mpimg.imread(images[0])
cal_undst = cv2.undistort(calimg, mtx, dist, None, mtx)
testimg=mpimg.imread('./test_images/test1.jpg')
test_undst = cv2.undistort(testimg, mtx, dist, None, mtx)


## Switch to inline plotting

%matplotlib inline

f1, ax1 = plt.subplots(2, 2,figsize=(15,10));
ax1[0,0].imshow(calimg)
ax1[0,0].set_title('Original Cal Image',fontsize=20);
ax1[0,1].imshow(cal_undst)
ax1[0,1].set_title('Undistorted Cal Image',fontsize=20);
ax1[1,0].imshow(testimg)
ax1[1,0].set_title('Original Test Image',fontsize=20);
ax1[1,1].imshow(test_undst)
ax1[1,1].set_title('Undistorted Test Image',fontsize=20);


This appears to work as expected.  On to thresholding. 

### Step 2. Colorspace and gradient thresholding to produce binary output.

For this I intend to use all of the test images but first I will define each function individually and test on a single test image to verify function. 

In previous projects and lessons, the YUV and LAB colorspaces showed strenghts in identifying features. I am going to create helper functions for that as well. The HLS mentioned in this lesson is included as well. 

In [ ]:
import matplotlib.gridspec as gridspec

def yuv_chn(img,channelIndex):
    tgt_channel = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)[:, :, channelIndex]
    return tgt_channel

def lab_chn(img,channelIndex):
    tgt_channel = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)[:, :, channelIndex]
    return tgt_channel

def hsv_chn(img,channelIndex):
    tgt_channel = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)[:, :, channelIndex]
    return tgt_channel

def hls_chn(img,channelIndex):
    tgt_channel = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)[:, :, channelIndex]
    return tgt_channel

def rgb_chn(img,channelIndex):
    tgt_channel = img[:, :, channelIndex]
    return tgt_channel

def plot_test_group(imgfunc,labellist,testset):
    f,ax=plt.subplots(len(testset),4,figsize=(17,20))
    for j in range(len(testset)):
        test_img=mpimg.imread(testset[j])
        ax[j,0].imshow(imgfunc(test_img,0),cmap='gray')
        ax[j,0].set_title(labellist[0])
        ax[j,0].axis('off')
        ax[j,1].imshow(imgfunc(test_img,1),cmap='gray')
        ax[j,1].set_title(labellist[1])
        ax[j,1].axis('off')
        ax[j,2].imshow(imgfunc(test_img,2),cmap='gray')
        ax[j,2].set_title(labellist[2])
        ax[j,2].axis('off')
        ax[j,3].imshow(test_img)
        ax[j,3].set_title('Original')
        ax[j,3].axis('off')
    f.subplots_adjust(wspace=0, hspace=0)

testset=glob.glob('./test_images/test*.jpg')

In [ ]:
yuvlabels=['Y Channel','U Channel','V Channel']
plot_test_group(yuv_chn,yuvlabels,testset)

## WRITE THOUGHTS ON YUV RESULTS

In [ ]:
lablabels=['L Channel','A Channel','B Channel']
plot_test_group(lab_chn,lablabels,testset)

## WRITE THOUGHTS ON LAB RESULTS

In [ ]:
rgblabels=['R Channel','G Channel','B Channel']
plot_test_group(rgb_chn,rgblabels,testset)

## WRITE THOUGHTS ON RGB RESULTS

In [ ]:
hsvlabels=['H Channel','S Channel','V Channel']
plot_test_group(hsv_chn,hsvlabels,testset)

## WRITE THOUGHTS ON HSV RESULTS

In [ ]:
hlslabels=['H Channel','L Channel','S Chanel']
plot_test_group(hls_chn,hslabels,testset)

In [ ]:
## Taking a hint from the lesson I will create the s channel function from the HLS colorspace. 

def s_chn_bin(img,thresh=(0,255)):
    s_channel = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)[:, :, 2]
    bin_out=np.zeros_like(s_channel)
    bin_out[(s_channel >= thresh[0]) & (s_channel <= thresh[1])] = 1
    return bin_out

## Test functions using previously loaded image
plt.imshow(s_chn_bin(testimg,(20,50)),cmap='gray');



### Create a function to apply thresholds to binary images. 

def applythresh(single_chan_img,thresh=(0,255)):
    bin_out=np.zeros_like(single_chan_img)
    bin_out[(single_chan_img >= thresh[0]) & (single_chan_img <= thresh[1])] = 1
    return bin_out